In [1]:
!pip install emoji
!python -m spacy download en_core_web_md
!ls

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')
sample_data


In [2]:
from google.colab import drive
drive.mount('/content/drive')
#!ls "/content/drive/My Drive/collab_sandbox/"
%cd drive/MyDrive/collab_sandbox/DataLabeling/
!ls

Mounted at /content/drive
/content/drive/MyDrive/collab_sandbox/DataLabeling
balanced_combined_texts.csv  export-2021-01-16T20_20_11.919Z.csv
combined_texts.csv	     Harry.txt
data_check.ipynb	     medium-for-grammar.csv
DataLabeling.zip	     news.csv
DataLabel.ipynb		     reddit_comments_positive.csv
documentation.txt	     ted-dataframe.csv


In [3]:
import pandas as pd
import spacy
import json
import re
import spacy
import emoji
nlp = spacy.load("en_core_web_md")

In [ ]:
# - Медиум
# - Гарри поттер
# - Документация фреймворка
# - Статьи с реддита
# - Новости с BBC
# - TED

In [4]:
def pretty_text(dirty_text):
    bad_chars = {
        "’":"'",
        "‘": "'",
        "“": ' ',
        "”": ' ',
        "—": "-",
        "…": "...",
        "–": "-",
        '"': " ",
        '[': "",
        "]": "",
        '(Laughter)': ' ',
        '(Applause)': ' ',
        '--': '-'
    }
    new_text = str(dirty_text).strip()
    for bad_char in bad_chars:
        new_text = new_text.replace(bad_char, bad_chars[bad_char])
                
    _replace_whitespace_ = re.compile(r"\s+")
    new_text = _replace_whitespace_.sub(" ", new_text).strip()

    _replace_new_chapter = re.compile(r"(Page \| [\d]{1,} Harry Potter and the Philosophers Stone - J.K. Rowling)")
    new_text = _replace_new_chapter.sub(" ", new_text).strip()

    _replace_multiple_whitespaces = re.compile(r'\s{1,}')
    new_text = _replace_multiple_whitespaces.sub(" ", new_text).strip()

    _replace_any_link_ = re.compile(r"http[^\s]+")
    new_text = _replace_any_link_.sub(" ", new_text).strip()

    return new_text

# - Медиум

In [5]:
medium_dataset = pd.read_csv("medium-for-grammar.csv", encoding='utf-8')
medium_dataset = medium_dataset.drop(columns=['title', 'img', 'link', 'topic', 'uuid'])

# - Гарри поттер

In [ ]:
for item in {"qwe":12, 'asd':3 }:
    print(item)

qwe
asd


In [6]:
harry_dataset = open("Harry.txt", encoding='utf-8').read()
harry_dataset = pretty_text(harry_dataset)

## - Документация фреймворка

In [7]:
documentations_dataset = open("documentation.txt", encoding='utf-8').read()
documentations_dataset = pretty_text(documentations_dataset)

## - Статьи с реддита

In [8]:
reddit_dataset = pd.read_csv("reddit_comments_positive.csv", encoding='utf-8')
reddit_dataset = reddit_dataset.iloc[:5000]
reddit_dataset = reddit_dataset.drop(columns=['id', 'parent_id', 
                                    'subreddit_id', 
                                    'link_id', 
                                    'score',
                                    'ups',
                                    'author',
                                    'controversiality',
                                    'parent_link_id',
                                    'parent_score',
                                    'parent_ups',
                                    'parent_author',
                                    'parent_controversiality'
                                    ])
reddit_dataset.head()

,text,parent_text
0,This isn't Twitter: try to comment on the arti...,Fucking faggot.
1,"Well, it is exactly what it sounds like. It's ...","Elaborate on this cum box, please."
2,"In soviet Russia, bomb disarms you!","I don't live in Russia anymore, and I will not..."
3,"""runin for senitur! #YOLO!""",This just made me realize that future presiden...
4,You step motherfucker.,I have sex with my step mom when my dad isn't ...


In [9]:
def convert_reddit(row):
    parent_text = row['parent_text']
    text = row['text']
    result = pretty_text(text) + " " + pretty_text(parent_text)
    return result

convert_reddit(reddit_dataset.iloc[24])

'I asked my Russian friend this once. He told me he dreams in English because my mom only speaks English and his dreams are exclusively of fucking my mom. Your answer was far more interesting. I dream in english. That actually happened, later in life, but now I dream in english. When I learned to speak english fluently, my dreams changed to english. Strange, *right?*'

## - Новости 

In [27]:
news_dataset = pd.read_csv("news.csv", encoding='utf-8')
news_dataset = news_dataset.drop(columns=[
                                    'id', 
                                    'title', 
                                    'publication', 
                                    'author',
                                    'date',
                                    'year',
                                    'month',
                                    'url',
                                    'Unnamed: 0'
                                    ])
news_dataset.head()

,content
0,The son of a Louisiana man whose father was sh...
1,Copies of William Shakespeare’s first four boo...
2,"Debt: $20, 000, Source: College, credit cards,..."
3,"It was late. I was drunk, nearing my 35th birt..."
4,A central Texas man serving a life sentence fo...


## - TED

In [11]:
ted_dataset = pd.read_csv("ted-dataframe.csv", encoding='utf-8')
ted_dataset = ted_dataset.drop(columns=[
                                    'talk_name', 
                                    'talk_url', 
                                    'talk_url_transcript', 
                                    'ted_number',
                                    ])
ted_dataset.head()

,text
0,"Thank you so much, Chris. And it's truly a gre..."
1,"In terms of invention, I'd like to tell you th..."
2,"A public, Dewey long ago observed, is constitu..."
3,"I want to start off by saying, Houston, we hav..."
4,"What I want to talk about is, as background, i..."


## Create dataset

In [23]:
texts_dataframe = pd.DataFrame(columns=['sentence', 'source'])

In [ ]:
all_texts = {
    'news': news_dataset,
    'ted': ted_dataset,
    'reddit': reddit_dataset,
    'harry': harry_dataset,
    'medium': medium_dataset,
    'documentation': documentations_dataset
}

In [13]:
def sent_tokinizer(text, max_length=40, min_length=12):
    text = str(text)
    doc = nlp(text)
    sentences = []
    for sent in doc.sents:
        sent = str(sent)
        doc_sent = nlp(sent)
        if (len(doc_sent) > min_length 
        and len(doc_sent) < max_length 
        and sent[0].isalpha() 
        and sent[0].isupper()  
        and not '...' in sent 
        and not bool(emoji.get_emoji_regexp().search(sent))):
            # print(f"{len(doc_sent)} - {sent}")
            sentences.append(sent)
    return sentences

In [28]:
news_dataset = news_dataset['content'].apply(pretty_text)
len(news_dataset)

42571

In [15]:
ted_dataset = ted_dataset['text'].apply(pretty_text)

In [16]:
reddit_dataset = [convert_reddit(reddit_dataset.iloc[i]) for i in range(4999)]

In [17]:
medium_dataset = medium_dataset['text'].apply(pretty_text)

In [ ]:
# sent_tokinizer(medium_dataset.iloc[2])

In [ ]:
# sent_tokinizer(news_dataset.iloc[50])

In [18]:
sent_tokinizer(ted_dataset.iloc[0])

["And it's truly a great honor to have the opportunity to come to this stage twice; I'm extremely grateful.",
 'I have been blown away by this conference, and I want to thank all of you for the many nice comments about what I had to say the other night.',
 'And I say that sincerely, partly because (Mock sob) I need that.',
 'Now I have to take off my shoes or boots to get on an airplane!',
 "I'll tell you one quick story to illustrate what that's been like for me.",
 "It's a true story - every bit of this is true.",
 'White House - we were driving from our home in Nashville to a little farm we have 50 miles east of Nashville.',
 'I know it sounds like a little thing to you, but - I looked in the rear-view mirror and all of a sudden it just hit me.',
 'It was dinnertime, and we started looking for a place to eat.',
 "We got off the exit, we found a Shoney's restaurant.",
 "Low-cost family restaurant chain, for those of you who don't know it.",
 'We went in and sat down at the booth, and

In [ ]:
# sent_tokinizer(reddit_dataset[24])

['He told me he dreams in English because my mom only speaks English and his dreams are exclusively of fucking my mom.',
 'That actually happened, later in life, but now I dream in english.',
 'When I learned to speak english fluently, my dreams changed to english.']

In [ ]:
 '...' in ''

False

## Add news

In [29]:
for i in range(300):
    text = news_dataset.iloc[i]
    sents = sent_tokinizer(text)
    if len(sents) > 0:
        for sent in sents:
            row = {}
            row['sentence'] = sent
            row['source'] = 'news'
            texts_dataframe = texts_dataframe.append(row, ignore_index=True)

In [30]:
len(texts_dataframe)

36812

## Add Ted

In [31]:
for i in range(150):
    text = ted_dataset.iloc[i]
    sents = sent_tokinizer(text)
    if len(sents) > 0:
        for sent in sents:
            row = {}
            row['sentence'] = sent
            row['source'] = 'ted'
            texts_dataframe = texts_dataframe.append(row, ignore_index=True)

In [32]:
len(texts_dataframe)

50001

## Add Reddit

In [33]:
for i in range(1500):
    text = reddit_dataset[i]
    sents = sent_tokinizer(text)
    if len(sents) > 0:
        for sent in sents:
            row = {}
            row['sentence'] = sent
            row['source'] = 'reddit'
            texts_dataframe = texts_dataframe.append(row, ignore_index=True)

In [34]:
len(texts_dataframe)

52640

## Add harry 

In [35]:
harry_sents = sent_tokinizer(harry_dataset)
if len(harry_sents) > 0:
    for sent in harry_sents:
        row = {}
        row['sentence'] = sent
        row['source'] = 'harry'
        texts_dataframe = texts_dataframe.append(row, ignore_index=True)

In [36]:
len(texts_dataframe)

55279

## Add medium

In [37]:
len(medium_dataset)

115

In [38]:
for i in range(100):
    text = medium_dataset.iloc[i]
    sents = sent_tokinizer(text)
    if len(sents) > 0:
        for sent in sents:
            row = {}
            row['sentence'] = sent
            row['source'] = 'medium'
            texts_dataframe = texts_dataframe.append(row, ignore_index=True)

## Add documentations_dataset

In [39]:
document_sents = sent_tokinizer(documentations_dataset)
if len(harry_sents) > 0:
    for sent in document_sents:
        row = {}
        row['sentence'] = sent
        row['source'] = 'document'
        texts_dataframe = texts_dataframe.append(row, ignore_index=True)

In [40]:
len(texts_dataframe)

59552

In [41]:
texts_dataframe.to_csv('combined_texts.csv', encoding='utf-8', index=False)

In [42]:
texts_dataframe

,sentence,source
0,The son of a Louisiana man whose father was sh...,news
1,"Cameron Sterling, the son of Alton Sterling, w...",news
2,Alton Sterling was killed by Baton Rouge polic...,news
3,Baton Rouge police said in a statement that po...,news
4,The press conference on Wednesday is Cameron's...,news
...,...,...
59547,"This can be useful in many cases, including el...",document
59548,If there's a single input to an operation that...,document
59549,"Conversely, only if all inputs don't require g...",document
59550,Backward computation is never performed in the...,document


In [ ]:
800 / 6

133.33333333333334

# Split combined data

In [ ]:
combined_texts = pd.read_csv('combined_texts.csv', encoding='utf-8')

In [ ]:
combined_texts[combined_texts['source'] == 'ted'].iloc[:]

,sentence,source
1450,And it's truly a great honor to have the oppor...,ted
1451,"I have been blown away by this conference, and...",ted
1452,Now I have to take off my shoes or boots to ge...,ted
1453,I'll tell you one quick story to illustrate wh...,ted
1454,It's a true story - every bit of this is true.,ted
...,...,...
6092,I just should note: I have two laptops and one...,ted
6093,So it's important to have these proven ideas p...,ted
6094,"My mom once said, There's nothing worse than b...",ted
6095,We've changed FEMA guidelines; we've changed p...,ted


In [ ]:
texts_domains = ['news', 'reddit', 'document', 'harry', 'medium', 'ted']
balanced_combined_texts = pd.DataFrame(columns=['sentence', 'source'])
for sub_domain in texts_domains:
    sub_dataframe = combined_texts[combined_texts['source'] == sub_domain].sample(n=154, random_state=1, replace=True)
    balanced_combined_texts = pd.concat([balanced_combined_texts, sub_dataframe])

In [ ]:
balanced_combined_texts.to_csv('balanced_combined_texts.csv', encoding='utf-8', index=False)

In [ ]:
len(balanced_combined_texts)

924